In [2]:
#from Limousine_central import*
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [ ]:

ta= pd.read_csv('/Users/celinej/Documents/Mes_notebooks/Limousine/lecture tables/title.akas.tsv.gz', sep='\t', header=0, low_memory=False, na_values= ['\\N'])
nb= pd.read_csv('/Users/celinej/Documents/Mes_notebooks/Limousine/lecture tables/name.basics.tsv.gz', sep='\t', header=0,low_memory=False, na_values= ['\\N'])
tb= pd.read_csv('/Users/celinej/Documents/Mes_notebooks/Limousine/lecture tables/title.basics.tsv.gz', sep='\t', header=0,low_memory=False, na_values= ['\\N'])
#tc= pd.read_csv('/Users/celinejoret/Documents/Mes_notebooks/Limousine/lecture tables/title.crew.tsv.gz', sep='\t', header=0,low_memory=False, na_values= ['\\N'])
tp= pd.read_csv('/Users/celinej/Documents/Mes_notebooks/Limousine/lecture tables/title.principals.tsv.gz', sep='\t', header=0,low_memory=False, na_values= ['\\N'])
tr= pd.read_csv('/Users/celinej/Documents/Mes_notebooks/Limousine/lecture tables/title.ratings.tsv.gz', sep='\t', header=0,low_memory=False, na_values= ['\\N'])



Travail sur la table Akas (copies internationales)

Table de plus de 31 millions de lignes.

Nous effectuons des tris de façon à ne garder qu'une ligne par oeuvre si possible dans sa version de titre en Français.

Nous supprimons les colonnes inutiles et remplaçons les Nan région et langue quand c'est possible.

Il reste au final 6 millions de lignes.



In [4]:
ta.drop(['types', 'attributes'], axis= 1, inplace= True)

In [5]:
ta1= ta[(ta['language']== 'fr') | (ta['language'].isna())]

In [6]:
ta_FR_lg_Nan= ta1[(ta1['region']== 'FR') & (ta1['language'].isna())]

In [ ]:

ta_FR_lg_Nan['language'].fillna('fr', inplace=True)

In [8]:
ta_non_FR=ta1[ ~(ta1['region']=='FR')]

In [ ]:
ta_non_FR.sort_values(by=['titleId','language'], ascending=[True, False], inplace= False)

In [11]:
ta_FR_non_Nan=ta1[ (ta1['region']=='FR') & (ta1['language'].notna()) ]

In [12]:

ta1= pd.concat([ta_FR_lg_Nan,ta_non_FR, ta_FR_non_Nan ])

ta1.sort_values(by=['titleId','language'], ascending=[True, False], inplace= True)
ta1.drop_duplicates(subset= 'titleId',  keep='first', inplace= True)

Travail sur la table title ratting (votes)

1.2 millions de lignes.

Nous avons décider de ne garder que les oeuvres les plus populaires.

Soit ayant plus de 20 000 votes (les blockbusters);

Soit ayant plus de 1 000 votes et une note suppérieure à 7.5

Ce tri représente 32 200 oeuvres.




In [ ]:
tr1= tr[ 
        (tr["numVotes"]>20000)
        |
        (tr["numVotes"]>1000) & (tr['averageRating']>7.5) ]
tr1.shape

Travail sur la table "title basics". (description de chaque oeuvre)

8.8 millions de lignes.

Après traitement des Nan.

Nous décidons de ne garder que:

les films (comprend documentaires et films d'animation)

les oeuvres de plus de 70 mn et moins de 300 mn

les oeuvres depuis 1960
    

Il reste alors 444 000 oeuvres.

In [15]:

tb['runtimeMinutes'] =pd.to_numeric(tb['runtimeMinutes'], errors='coerce', downcast=None)

tb_t1=tb[ 
    ( (tb['runtimeMinutes']> 70) & (tb['runtimeMinutes'] < 300)  | (tb['runtimeMinutes'].isna()) )
    & 
    (tb['titleType']=='movie')
    & 
    ((tb['startYear']> 1960)| (tb['startYear'].isna()))
    ]

On fusionne les 3 tables akas- 

title basics- title rating

Il reste 8838 films

In [19]:
ta_tb = pd.merge(ta1,
                  tb_t1,
                  how='inner',
                  left_on="titleId",
                  right_on= "tconst")

#merge title rating avec ta_tb =tt

tt = pd.merge(ta_tb,
                  tr1,
                  how='inner',
                  left_on="titleId",
                  right_on= "tconst")

In [21]:
tt.head()

,titleId,ordering,title,region,language,isOriginalTitle,tconst_x,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tconst_y,averageRating,numVotes
0,tt0035423,10,Kate et Léopold,FR,fr,0.0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0.0,2001.0,NaN,118.0,"Comedy,Fantasy,Romance",tt0035423,6.4,83472
1,tt0054130,34,La Nuit,FR,fr,0.0,tt0054130,movie,La Notte,La notte,0.0,1961.0,NaN,122.0,Drama,tt0054130,8.0,20682
2,tt0054599,2,Accatone,FR,fr,0.0,tt0054599,movie,Accattone,Accattone,0.0,1961.0,NaN,117.0,Drama,tt0054599,7.7,8617
3,tt0054632,27,L'année dernière à Marienbad,FR,fr,0.0,tt0054632,movie,Last Year at Marienbad,L'année dernière à Marienbad,0.0,1961.0,NaN,94.0,"Drama,Mystery",tt0054632,7.7,22266
4,tt0054665,13,Le baron de crac,FR,fr,0.0,tt0054665,movie,The Fabulous Baron Munchausen,Baron Prásil,0.0,1962.0,NaN,83.0,"Adventure,Animation,Comedy",tt0054665,7.7,2472


Nous effectuons un tri supplémentaire 

Les films avant 1984 doivent avoir plus de 3000 votes et une moyenne supérieure à 9.

Il nous reste 7795 films

In [22]:
tt_t1 = tt[ 
                ( (tt['startYear'] < 1984) & (tt['averageRating'] >= 9) &  (tt["numVotes"]>3000)) 
                |
                (tt['startYear'] >= 1984) ]

In [ ]:

tt_t1.drop(['isOriginalTitle', 'ordering','tconst_x','titleType','primaryTitle','originalTitle','endYear', 'tconst_y','isAdult' ], axis= 1, inplace= True)

Travail sur title principal (intervenants principaux dans chaque oeuvre)

50 millions de lignes

Nous ne gardons que les lignes qui concernent les acteurs, les actrices et les réalisateurs.

Il reste alors 25 millions de lignes

In [30]:
mask = tp.drop(columns = ['ordering', 'job','characters' ])
maskbis = mask['category'].str.contains('composer|editor|production_designer|archive_footage|cinematographer|archive_sound|writer|producer|self')
index_to_drop = mask[maskbis].index
mask.drop(index_to_drop , inplace=True)
tp1= mask.copy()

In [31]:
tp1['category'].unique()

array(['director', 'actor', 'actress'], dtype=object)

In [32]:
tp1.shape

(25359003, 3)

code pour récup des infos sur API

In [ ]:
import aiohttp
import asyncio

In [ ]:
# TCONST

async def get_tconst(session, url, tc):
    async with session.get(url) as resp:
        result = await resp.json()
        title = result['movie_results'][0]['title']
        poster = result['movie_results'][0]['poster_path']
        overview = result['movie_results'][0]['overview']
        return {'tc': tc,
                'result': result['movie_results'][0],
                'title': title, 
                'poster': f"https://image.tmdb.org/t/p/original%7Bposter%7D", 
                'overview': overview
                }

async def get_tconsts_infos(tconsts):

    async with aiohttp.ClientSession() as session:

        tasks = []
        for tc in tconsts:
            url = f'https://api.themoviedb.org/3/find/%7B tc }?api_key={d1a8eba8177a6e156ffa01ca145aaa7f}&external_source=imdb_id&language=fr-FR'
            tasks.append(asyncio.ensure_future(get_tconst(session, url, tc)))

        movies = {}
        original_tconst = await asyncio.gather(tasks)
        for tc in original_tconst:
            movies[tc['tc']] = tc

        return movies



On fusionne la table title principal avec la table générale que nous avons obtenu précédemment.

Nous avons 37 355 lignes

In [33]:
tt_tp= pd.merge(tt_t1,
        tp1,
        how='inner',
        left_on="titleId",
        right_on= "tconst")

In [36]:
tt_tp.drop(['titleId'], axis= 1, inplace= True)

In [37]:
tt_tp.head(10)

,title,region,language,startYear,runtimeMinutes,genres,averageRating,numVotes,tconst,nconst,category
0,Kate et Léopold,FR,fr,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000212,actress
1,Kate et Léopold,FR,fr,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0413168,actor
2,Kate et Léopold,FR,fr,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000630,actor
3,Kate et Léopold,FR,fr,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0005227,actor
4,Kate et Léopold,FR,fr,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0003506,director
5,La vérification,FR,fr,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0125909,actor
6,La vérification,FR,fr,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0813463,actor
7,La vérification,FR,fr,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0952519,actor
8,La vérification,FR,fr,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0096846,actor
9,La vérification,FR,fr,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0314516,director


Travail sur la table Name Basics (descriptif des individus)

Pour éliminer les outliers on ne garde que les personnes nées après 1853.

On ne conserve que les personnes dont le travail est acteur, actrice ou réalisateur.


In [ ]:
nb = nb.rename(columns={'primaryName': 'pname','birthYear': 'byear', 'deathYear': 'dyear', 'primaryProfession': 'fjob', 'knownForTitles': 'kftitles'})


nb['byear'] = nb['byear'].fillna(1500).astype(int)
nb['dyear'] = nb['dyear'].fillna(1500).astype(int)

mask_byear = nb['byear'] > 1853
nb = nb[mask_byear]

mask = nb['byear'] > nb['dyear']
nb[mask]

In [ ]:
tmp = []
for i in range(len(nb['fjob'])):
    s = str(nb['fjob'].iloc[i])
    tmp.extend(s.split(','))
tmp = set(tmp)

tmp = pd.DataFrame(tmp)
tmp

In [40]:

pertinent_job = nb['fjob'].str.contains('actor|actress|director').fillna(False)
nb = nb[pertinent_job]

nb_copy = nb

In [41]:
nb_copy.shape

(380249, 6)

On fusionne la table "name basic" avec notre grosse table précédente.
Il nous reste environ 30 000 lignes

In [277]:
#Merge name basics avec tt_tp

tt_tb= pd.merge(tt_tp,
        nb_copy,
        how='inner',
        left_on="nconst",
        right_on= "nconst")

In [278]:
tt_tb.shape

(30454, 16)

In [282]:
tt_tb.drop([ 'dyear','language'], axis= 1, inplace= True)

In [284]:
tt_tb.sort_values('tconst', inplace=True)

In [285]:
tt_tb.head(30)

,title,region,startYear,runtimeMinutes,genres,averageRating,numVotes,tconst,nconst,category,pname,byear,fjob,kftitles
0,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000212,actress,Meg Ryan,1961,"actress,producer,soundtrack","tt0128853,tt0108160,tt0120632,tt0098635"
70,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0003506,director,James Mangold,1963,"producer,director,writer","tt0113280,tt1950186,tt3315342,tt0358273"
64,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0005227,actor,Breckin Meyer,1974,"actor,writer,producer","tt0035423,tt0112697,tt0215129,tt0250687"
47,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000630,actor,Liev Schreiber,1967,"actor,producer,miscellaneous","tt1895587,tt0404030,tt0368008,tt0458525"
18,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0413168,actor,Hugh Jackman,1968,"actor,soundtrack,producer","tt0120903,tt3315342,tt1707386,tt0458525"
80,La vérification,FR,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0125909,actor,Rolan Bykov,1929,"actor,director,writer","tt0060107,tt0061876,tt0060083,tt0085335"
84,La vérification,FR,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0952519,actor,Vladimir Zamanskiy,1928,actor,"tt5077564,tt0095040,tt0312341,tt0124127"
83,La vérification,FR,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0813463,actor,Anatoliy Solonitsyn,1934,actor,"tt0069293,tt0072443,tt0079944,tt0060107"
86,La vérification,FR,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0314516,director,Aleksey German,1938,"actor,writer,director","tt2328813,tt0156701,tt0084345,tt0074447"
85,La vérification,FR,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0096846,actor,Oleg Borisov,1929,"actor,director","tt0096121,tt0100297,tt0104370,tt0076954"


In [286]:
tt_tb.shape

(30454, 14)

In [287]:
tt_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30454 entries, 0 to 22545
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           30454 non-null  object 
 1   region          29605 non-null  object 
 2   startYear       30454 non-null  float64
 3   runtimeMinutes  30450 non-null  float64
 4   genres          30454 non-null  object 
 5   averageRating   30454 non-null  float64
 6   numVotes        30454 non-null  int64  
 7   tconst          30454 non-null  object 
 8   nconst          30454 non-null  object 
 9   category        30454 non-null  object 
 10  pname           30454 non-null  object 
 11  byear           30454 non-null  int64  
 12  fjob            30454 non-null  object 
 13  kftitles        30454 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 3.5+ MB


averagerating/startYear
region / averageRating
genre/averageRating
actor/actress

In [288]:
df= tt_tb.copy()

In [289]:
df.dropna(axis=0, inplace=True)

Pour appliquer la méthode des KNN nous avons préparé notre base de données:

traitement de tous les Nan

Remplacement de chaque variable type objet par une variable numérique (utilisation de factorize et de getdummies).

Requêté les API d'IMDB pour renseigné la région d'origine des films, les titres des versions Françaises et le résumé

Utilisé le coefficiant de recommandation utilisé par IMDB (WR)

In [ ]:
# Create genres list
my_set = set()
my_list = ['tconst']

for i in df['genres']:
    my_set.add(i)
for i in my_set:
    my_list.extend(i.split(','))
res = list(set(my_list))

# Create a DataFrame with the genre list and fill it with tconst & genres values
genre_tab = pd.DataFrame(columns=res)
genre_tab['tconst'] = df['tconst']
genre_tab['genres'] = df['genres']
genre_tab = genre_tab.set_index('tconst')

# Fill with 1 each columns
for i in genre_tab.columns:
    if i != 'genres':
        genre_tab[i][genre_tab['genres'].str.contains(i)] = 1
        genre_tab[i] = genre_tab[i].fillna(0)
genre_tab.drop(columns='genres', inplace=True)


In [292]:
genre_tab.shape

(29605, 22)

In [293]:
genre_tab.head(20)

,Thriller,History,Comedy,Romance,Western,Sci-Fi,News,Family,Drama,Musical,Biography,Adventure,Horror,War,Animation,Action,Sport,Music,Documentary,Crime,Mystery,Fantasy
tconst,,,,,,,,,,,,,,,,,,,,,,
tt0035423,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
tt0035423,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
tt0035423,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
tt0035423,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
tt0035423,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
tt0067625,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
tt0067625,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
tt0067625,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
tt0067625,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [294]:
df.shape

(29605, 14)

In [295]:
#on merge la table des genres avec le df général
df_ml= pd.merge(df,
       genre_tab,
        how='inner',
        left_on="tconst",
        right_on= "tconst")

In [296]:
df_ml.shape

(136451, 36)

In [297]:
df_ml.drop_duplicates(inplace=True)

In [301]:
df_ml.category.unique()

array(['actress', 'director', 'actor'], dtype=object)

In [303]:
df_ml['region_num']= df_ml['region'].factorize()[0]

In [304]:
df_ml['nconst_num']= df_ml['nconst'].factorize()[0]

In [305]:
df_ml['category_num']= df_ml['category'].factorize()[0]

In [ ]:
df_ml.drop(columns=['genres', 'category', 'kftitles'])

In [308]:
df_ml.to_csv('df_jobs.csv', index=False)

In [268]:
df_ml.reset_index(drop=True, inplace=True)

In [ ]:
m= df_ml['numVotes'].min()
C= df_ml['averageRating'].mean()

def WR(x, m=m, C=C):
    v = df_ml['numVotes']
    R = df_ml['averageRating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)


df_ml['WR']= WR(df_ml)

In [10]:
df_ml= pd.read_csv('lecture tables/df_ml.csv')
df_ml

,title,region,startYear,runtimeMinutes,genres,averageRating,numVotes,tconst,nconst,category,pname,byear,kftitles,Thriller,History,Comedy,Romance,Western,Sci-Fi,News,Family,Drama,Musical,Biography,Adventure,Horror,War,Animation,Action,Sport,Music,Documentary,Crime,Mystery,Fantasy,region_num,nconst_num,category_num,WR
0,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000212,actress,Meg Ryan,1961,"tt0128853,tt0108160,tt0120632,tt0098635",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,6.403932
1,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0003506,director,James Mangold,1963,"tt0113280,tt1950186,tt3315342,tt0358273",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,6.403932
2,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0005227,actor,Breckin Meyer,1974,"tt0035423,tt0112697,tt0215129,tt0250687",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,2,6.403932
3,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000630,actor,Liev Schreiber,1967,"tt1895587,tt0404030,tt0368008,tt0458525",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,2,6.403932
4,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0413168,actor,Hugh Jackman,1968,"tt0120903,tt3315342,tt1707386,tt0458525",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,2,6.403932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29600,I Care a Lot,FR,2020.0,118.0,"Comedy,Crime,Drama",6.3,127033,tt9893250,nm0227759,actor,Peter Dinklage,1969,"tt0944947,tt1877832,tt0340377,tt5027774",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4921,2,6.303380
29601,I Care a Lot,FR,2020.0,118.0,"Comedy,Crime,Drama",6.3,127033,tt9893250,nm2555462,actress,Eiza González,1990,"tt9893250,tt3890160,tt5034838,tt6806448",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,8066,0,6.303380
29602,I Care a Lot,FR,2020.0,118.0,"Comedy,Crime,Drama",6.3,127033,tt9893250,nm0683253,actress,Rosamund Pike,1979,"tt0246460,tt0790724,tt0414387,tt2267998",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4128,0,6.303380
29603,I Care a Lot,FR,2020.0,118.0,"Comedy,Crime,Drama",6.3,127033,tt9893250,nm0001848,actress,Dianne Wiest,1946,"tt0099487,tt0109348,tt0091167,tt0115685",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,119,0,6.303380


Traitement sur la table pour obtenir la nationalité des films

In [ ]:
language ={'EN':'USA','RU':'RUS','ES':'ESP','TA':'IND','SL':'HUN','AF':'ZAF','LV':'LVA','PL':'POL',
'EL':'GRC','RO':'ROU','BS':'BIH','NO':'NOR','KY':'KGZ','CS':'CZE','FA':'IRN','NL':'NLD','FR':'FRA','PT':'BRA','KN':'IND','UK':'UKR','SH':'BIH','CN':'CHN','FI':'FIN','KO':'PRK','ET':'EST','HE':'ISR',
'SR':'BIH','SV':'SWE','BN':'BGD','UR':'PAK','GU':'IND','SQ':'ALB','ZH':'TWN','AR':'EGY','TE':'IND','PA':'IND','KK':'KAZ','HU':'HUN','ML':'DEU','BO':'USA','SI':'LKA','IS':'ETH','IT':'ITA','TR':'TUR',
'MK':'MKD','BG':'POL','LT':'RUS','JA':'JPN','DA':'DNK','SK':'CZE','ID':'USA','NB':'NOR','HI':'IND','DE':'DEU','HR':'ESP','TH':'THA','MR':'IND','KA':'GEO','XX':'CHE','BM':'BFA','TL':'GBR','USA':'USA'}

df_ml['alpha3']=df_ml.original_language.map(language)

In [ ]:
import pycountry_convert as pc

list1 =df2.alpha3.tolist()
list2=[]
list3=[]
for i in list1:
  try:
    list2.append(pc.country_alpha3_to_country_alpha2(i))
  except KeyError : 
    list2.append("NaN")
list2

for i in list2:
  try:
    list3.append(pc.country_alpha2_to_country_name(i,cn_name_format="default"))
  except KeyError : 
    list3.append("NaN")

dict={}


In [ ]:
df2.alpha3.value_counts().index.tolist()
list1=[]
list2=[]
for idx,name in enumerate(df2['alpha3'].value_counts().index.tolist()):
    list1.append(name)
    list2.append(df2['alpha3'].value_counts()[idx])
    
list1

dict={'alpha_code': list1,'count': list2}

df2.to_csv('df_alpha3', index= False)

In [6]:
df= pd.read_csv('lecture tables/df_alpha3.csv')
df

,Unnamed: 0,title,region,startYear,runtimeMinutes,genres,averageRating,numVotes,tconst,nconst,category,pname,byear,kftitles,Thriller,History,Comedy,Romance,Western,Sci-Fi,News,Family,Drama,Musical,Biography,Adventure,Horror,War,Animation,Action,Sport,Music,Documentary,Crime,Mystery,Fantasy,region_num,nconst_num,category_num,french_title,overview,original_language,alpha3
0,0,Kate et Léopold,FR,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,83472,tt0035423,nm0000212,actress,Meg Ryan,1961,"tt0128853,tt0108160,tt0120632,tt0098635",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Kate & Leopold,"Kate McKay est une femme d'affaires dynamique ; elle possède tous les atouts pour réussir sa vie professionnelle ; c'est une vraie business-woman du XXIe siècle. Leopold, troisième duc d'Albany, est un parfait gentleman, un séduisant célibataire... de la fin du XIXe siècle. La vie les a rendus cyniques : ils ne croient plus au grand amour. Mais quand une faille dans le cours du temps précipite Leopold dans le New York d'aujourd'hui, tout est prêt pour qu'ils s'aiment.",EN,USA
1,1,La vérification,FR,1986.0,96.0,"Drama,War",7.9,2257,tt0067625,nm0314516,director,Aleksey German,1938,"tt2328813,tt0156701,tt0084345,tt0074447",0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5,1,La Vérification,"Pendant l'hiver 1942, sous l'occupation nazie, Lazarev, passe à l'ennemi puis se rend aux partisans russes en prétendant avoir été pris malgré lui par les Allemands. Sa personnalité complexe oppose les deux chefs quant a sa sincérité.",RU,RUS
2,2,Le Parrain,FR,1972.0,175.0,"Crime,Drama",9.2,1771672,tt0068646,nm0000338,director,Francis Ford Coppola,1939,"tt0078788,tt0071562,tt0071360,tt0068646",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,10,1,Le Parrain,"La Seconde Guerre mondiale vient de s'achever. À New York, le « parrain » Don Corleone, l'un des chefs respectés de la mafia, se sent vieillir. Il refuse de s'adapter à son temps et de se lancer, comme ses pairs, dans le trafic de drogue. Une frilosité qui entrave la bonne marche des affaires des autres « familles » et qui lui vaut d'être la cible d'un attentat. Don Corleone survit à ses blessures, mais reste très diminué. Mike, son plus jeune fils, qui jusque-là se tenait à l'écart des affaires de son père, devient le plus dévoué de ses héritiers. Plus efficace que ses frères, Sonny et Fredo, il venge son père et organise l'élimination de ses adversaires…",EN,USA
3,3,"Le Parrain, 2ᵉ partie",FR,1974.0,202.0,"Crime,Drama",9.0,1225156,tt0071562,nm0000473,actress,Diane Keaton,1946,"tt0356680,tt0337741,tt0075686,tt0073312",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,13,0,"Le Parrain, 2e partie","Depuis la mort de Don Vito Corleone, son fils Michael règne sur la famille. Amené à négocier avec la mafia juive, il perd alors le soutien d'un de ses lieutenants, Frankie Pentageli. Échappant de justesse à un attentat, Michael tente de retrouver le coupable.",EN,USA
4,4,L'interrogatoire,FR,1989.0,118.0,"Crime,Drama,Thriller",8.0,3267,tt0084548,nm0002140,actress,Agnieszka Holland,1948,"tt0099776,tt0111507,tt6828390,tt5328350",1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,17,0,L'Interrogatoire,"Dans la Pologne stalinienne, la chanteuse de cabaret Tonia décide de passer la soirée à boire avec un groupe d'amis. Le lendemain matin elle se réveille en prison et découvre qu'elle a été arrêtée pour des raisons politiques. Alors que les responsables de la prison l'interrogent, la torturent et l'humilient, elle se bat pour sa survie et pour maintenir son innocence en refusant de signer de faux aveux. Au fil de ses années d'emprisonnement, sa relation avec ses ravisseurs se complique.",PL,POL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6926,6926,Hôtel Transylvanie 4: Changements monstres,FR,2022.0,87.0,"Adventure,Animation,Comedy",6.0,25165,tt9848626,nm0300712,actor,Jim Gaffigan,1966,"tt6769326,tt117674